# LAB 3: Frequency filtering 

## **Exercice 1**: 
>Given the Fourier Transform of an image, compute (a) the lowest and highest magnitude; (b) display a boxplot with the magnitude values; and (c) plot an histogram of the phase values. To display the boxplot and the histogram, you can use Matplotlib

## **Exercice 2**: 
>We have seen the convolution theorem applied to the mean filter. Now, apply it to the Gaussian filter and check the result is also as expected. In addition the (filtered) output image, display also the Fourier transform of the filter, and the element-wise product of the Fourier transforms of the image and the filter. Try to figure out the relationship between the size of the mean filter and its Fourier transform. Repeat this analysis on the standard deviation.

## **Exercice 3**: 
>A practical implication of the convolution theorem is the computational saving it may offer by performing computations in the frequency domain instead of in the spatial domain. Carry out a study of the running times for differnt sizes of the mask filter, and for different image sizes. Display a plot of the measured times (mean and deviation) to appreciate  the influence of the image size for a given mask size, and another plot where the image size is fixed and the mask size is varied. Observe the results carefully, and comment on them.

## **Exercice 4**: 
> Write `my_mask(n)` without the explicit loops that returns a 2D array of size $n·n$ following this pattern:
>
>$$
>M_{n=3} =
\begin{bmatrix}
    0 & -1 & -1 \\
    1 & 0 & -1 \\
    1 & 1 & 0
\end{bmatrix}
M_{n=4} =
\begin{bmatrix}
    0 & -1 & -1 & -1 \\
    1 & 0 & -1 & -1 \\
    1 & 1 & 0 & -1 \\
    1 & 1 & 1 & 0
\end{bmatrix}
>$$
>
>Then write `my_filter(im, n)` to use the output of `my_mask(n)` as a mask to filter the image `im` in the frequency domain
    


## **Exercice 5**: 
>Get the Fourier transform of stp1.gif and stp2.gif. Let $F1$ and $F2$ be these transforms. Compute and observe the inverse transform of their combination $λ·F1 +(1 − λ) · F2,λ∈[0,1]$. What property of the $FT$ does this result illustrate? 

## **Exercice 6**: 
>The  frequency  filters  we  have  seen  set  to  all  (1)  or  nothing (0) so that the desired frequency ranges (either high, low or band) are selected or removed, with abrupt changes. Write smoothed versions of the filters so that transition between 0 to 1 is somehow smooth. For the same values of $r$ and $R$, ompare the abrupt and smooth filters  and  comment  on  the  possible  influence on  the result.
